In [2]:
!pip install nashpy

In [3]:
import numpy as np
import random

from fractions import Fraction
from scipy.optimize import minimize
import nashpy as nash

In [4]:
# Nombre de stratégies (nombres possibles)
strategies = [1, 2, 3, 4, 5]
n = len(strategies)

# Matrices de paiement initialisées à 0
U1 = np.zeros((n, n))  # Gains du joueur 1
U2 = np.zeros((n, n))  # Gains du joueur 2

# Remplissage des matrices de paiement
for i, A in enumerate(strategies):
    for j, B in enumerate(strategies):
        if abs(A - B) == 1:  # Si les nombres sont consécutifs
            if A < B:
                U1[i, j] = A + B  # Joueur 1 gagne A + B
                U2[i, j] = 0       # Joueur 2 obtient 0
            else:
                U1[i, j] = 0       # Joueur 1 obtient 0
                U2[i, j] = A + B  # Joueur 2 gagne A + B
        else:  # Sinon, chacun gagne son propre nombre
            U1[i, j] = A
            U2[i, j] = B

# Affichage des matrices de jeu
print("Matrice de gains du Joueur 1 (U1) :")
print(U1)
print("\nMatrice de gains du Joueur 2 (U2) :")
print(U2)


Matrice de gains du Joueur 1 (U1) :
[[1. 3. 1. 1. 1.]
 [0. 2. 5. 2. 2.]
 [3. 0. 3. 7. 3.]
 [4. 4. 0. 4. 9.]
 [5. 5. 5. 0. 5.]]

Matrice de gains du Joueur 2 (U2) :
[[1. 0. 3. 4. 5.]
 [3. 2. 0. 4. 5.]
 [1. 5. 3. 0. 5.]
 [1. 2. 7. 4. 0.]
 [1. 2. 3. 9. 5.]]


In [5]:
# ========================
# Strategy Utilities
# ========================

def one_hot_encoding(choice, num_actions=5):
    """Returns a one-hot vector for the selected 1-based action."""
    vec = np.zeros(num_actions)
    vec[choice - 1] = 1
    return vec

def draw_from_distribution(elements, probabilities):
    """Draws a single element from a list based on probabilities."""
    return random.choices(elements, weights=probabilities, k=1)[0]

def print_mixed_strategies(strategy_vector, num_players):
    """Print each player’s mixed strategy in simplest fractional form."""
    strategies_per_player = len(strategy_vector) // num_players
    fracs = [Fraction(p).limit_denominator(1000) for p in strategy_vector]
    for p in range(num_players):
        start, end = p*strategies_per_player, (p+1)*strategies_per_player
        print(f"Player {p+1}:", fracs[start:end])

# ========================
# Game Model Functions
# ========================

def bilinear_payoff(p1_strat, payoff_matrix, p2_strat):
    """Compute p1_stratᵀ · payoff_matrix · p2_strat."""
    return p1_strat.T @ payoff_matrix @ p2_strat

def expected_payoff(opponent_strategy, payoff_matrix):
    """Expected payoff vector for each pure strategy against opponent."""
    return payoff_matrix @ opponent_strategy

def pure_best_response(opponent_strategy, payoff_matrix):
    """Return the pure (one-hot) strategy that maximizes expected payoff."""
    expected = expected_payoff(opponent_strategy, payoff_matrix)
    br = np.zeros_like(expected)
    br[np.argmax(expected)] = 1
    return br

def payoff_gradient(A, joint_strat, player_index):
    """Gradient of bilinear payoff for player_index in a 2-player game."""
    n = len(joint_strat) // 2
    x0, x1 = joint_strat[:n], joint_strat[n:]
    return (A @ x1) if player_index == 0 else (A.T @ x0)

def objective_function(x, payoff_matrices, num_players):
    """Sum of squared gradients; zero at a stationary (Nash) point."""
    err = 0
    for i in range(num_players):
        g = payoff_gradient(payoff_matrices[i], x, i)
        err += np.sum(g**2)
    return err

def stochastic_strategy_constraint(x, num_players):
    """Ensure each player’s probabilities sum to 1."""
    n = len(x) // num_players
    return np.array([np.sum(x[i*n:(i+1)*n]) - 1 for i in range(num_players)])

def hessian_of_payoff(_):
    """Hessian is zero in a bilinear game (placeholder)."""
    return 0

In [11]:
# ========================
# Game Definition & Solver
# ========================

num_players = 2
initial_strategy = np.ones(4) / 2  # 2 strategies per player

# Payoff matrices bataille des sexes
payoff_matrices = [
    np.array([[2, 0], [0, 1]]),  # Player 1
    np.array([[1, 0], [0, 2]])   # Player 2
]

# Constraints for SLSQP optimizer
constraints = [
    {'type': 'eq', 'fun': lambda x: stochastic_strategy_constraint(x, num_players)},
    {'type': 'ineq', 'fun': lambda x: x}  # Non-negative probabilities
]

# Solve the stationarity problem
solution_result = minimize(
    fun=objective_function,
    x0=initial_strategy,
    args=(payoff_matrices, num_players),
    method='SLSQP',
    constraints=constraints
)

# ========================
# Result Display
# ========================

if solution_result.success:
    strategy_solution = solution_result.x
    strat1,strat2=strategy_solution[:len(strategy_solution)//2],strategy_solution[len(strategy_solution)//2:]#Strategy of Players in equilibrium
    print("strat1",strat1)
    print("strat2",strat2)
    print("=== Stationary Strategy Found ===")
    print("Solution vector:", strategy_solution)
    print("Stationarity residual:", objective_function(strategy_solution, payoff_matrices, num_players))
    print("Hessian eigenvalue (should be 0):", hessian_of_payoff(payoff_matrices[0]))
    print("strat1",strat1)
    print("strat2",strat2)
    # NashPy enumeration
    game = nash.Game(payoff_matrices[0], payoff_matrices[1])
    equilibria = list(game.vertex_enumeration())
    print("stage_test 1 payoff vs stage_test 2")
    print(bilinear_payoff(strat1, payoff_matrices[0], strat2))
    print("stage_test1 vs stage_test2 payoff")
    print(bilinear_payoff(strat1, payoff_matrices[1], strat2))
    print("\n=== Nash Equilibria (via NashPy) ===")
    for eq in equilibria:
        print(f"Player 1: {eq[0]}")
        print(f"Player 2: {eq[1]}")
        print("Player1 payoff vs player2")#Payoff of player1 against player2
        print(bilinear_payoff(eq[0], payoff_matrices[0], eq[1]))
        print("Player1 vs player2 payoff")#Payoff of player2 against player1
        print(bilinear_payoff(eq[0], payoff_matrices[1], eq[1]))
        print("player 1 payoff vs stage_test2")
        print(bilinear_payoff(eq[0], payoff_matrices[0], strat2))
        print("player 1 vs stage_test2 payoff")
        print(bilinear_payoff(eq[0], payoff_matrices[1], strat2))

        print("stage_test1 payoff vs player2")
        print(bilinear_payoff(strat1, payoff_matrices[0], eq[1]))
        print("stage_test2 vs player2 payoff")
        print(bilinear_payoff(strat1, payoff_matrices[1], eq[1]))

else:
    print("❌ No feasible solution was found by the optimizer.")


strat1 [0.8 0.2]
strat2 [0.19999999 0.80000001]
=== Stationary Strategy Found ===
Solution vector: [0.8        0.2        0.19999999 0.80000001]
Stationarity residual: 1.6000000023841863
Hessian eigenvalue (should be 0): 0
strat1 [0.8 0.2]
strat2 [0.19999999 0.80000001]
stage_test 1 payoff vs stage_test 2
0.4799999862462297
stage_test1 vs stage_test2 payoff
0.47999999399483273

=== Nash Equilibria (via NashPy) ===
Player 1: [1. 0.]
Player 2: [ 1.00000000e+00 -2.22044605e-16]
Player1 payoff vs player2
2.0000000000000004
Player1 vs player2 payoff
1.0000000000000002
player 1 payoff vs stage_test2
0.3999999810755273
player 1 vs stage_test2 payoff
0.19999999053776366
stage_test1 payoff vs player2
1.6000000040233135
stage_test2 vs player2 payoff
0.8000000020116567
Player 1: [-2.22044605e-16  1.00000000e+00]
Player 2: [0. 1.]
Player1 payoff vs player2
1.0000000000000002
Player1 vs player2 payoff
2.0000000000000004
player 1 payoff vs stage_test2
0.8000000109523525
player 1 vs stage_test2 payof

In [12]:
# ========================
# Game Definition & Solver
# ========================

num_players = 2
initial_strategy = np.ones(4) / 2  # 2 strategies per player

# Payoff matrices Prisoners dilemma
payoff_matrices = [
    np.array([[3, 0], [5, 1]]),  # Player 1
    np.array([[3, 5], [0, 1]])   # Player 2
]

# Constraints for SLSQP optimizer
constraints = [
    {'type': 'eq', 'fun': lambda x: stochastic_strategy_constraint(x, num_players)},
    {'type': 'ineq', 'fun': lambda x: x}  # Non-negative probabilities
]

# Solve the stationarity problem
solution_result = minimize(
    fun=objective_function,
    x0=initial_strategy,
    args=(payoff_matrices, num_players),
    method='SLSQP',
    constraints=constraints
)

# ========================
# Result Display
# ========================

if solution_result.success:
    strategy_solution = solution_result.x
    strat1,strat2=strategy_solution[:len(strategy_solution)//2],strategy_solution[len(strategy_solution)//2:]
    print("=== Stationary Strategy Found ===")
    print("Solution vector:", strategy_solution)
    print("Stationarity residual:", objective_function(strategy_solution, payoff_matrices, num_players))
    print("Hessian eigenvalue (should be 0):", hessian_of_payoff(payoff_matrices[0]))
    print("strat1",strat1)
    print("strat2",strat2)
    # NashPy enumeration
    game = nash.Game(payoff_matrices[0], payoff_matrices[1])
    equilibria = list(game.vertex_enumeration())
    print("stage_test1 payoff vs stage_test2")
    print(bilinear_payoff(strat1, payoff_matrices[0], strat2))
    print("stage_test1 vs stage_test2 payoff")
    print(bilinear_payoff(strat1, payoff_matrices[1], strat2))
    print("\n=== Nash Equilibria (via NashPy) ===")
    for eq in equilibria:
        print(f"Player 1: {eq[0]}")
        print(f"Player 2: {eq[1]}")
        print("Player1 payoff vs player2")
        print(bilinear_payoff(eq[0], payoff_matrices[0], eq[1]))
        print("Player1 vs player2 payoff")
        print(bilinear_payoff(eq[0], payoff_matrices[1], eq[1]))
        print("player 1 payoff vs stage_test2")
        print(bilinear_payoff(eq[0], payoff_matrices[0], strat2))
        print("player 1 vs stage_test2 payoff")
        print(bilinear_payoff(eq[0], payoff_matrices[1], strat2))

        print("stage_test1 payoff vs player2")
        print(bilinear_payoff(strat1, payoff_matrices[0], eq[1]))
        print("stage_test1 vs player2 payoff")
        print(bilinear_payoff(strat1, payoff_matrices[1], eq[1]))

else:
    print("❌ No feasible solution was found by the optimizer.")

=== Stationary Strategy Found ===
Solution vector: [2.22311058e-12 1.00000000e+00 2.34590125e-12 1.00000000e+00]
Stationarity residual: 2.0000000000365525
Hessian eigenvalue (should be 0): 0
strat1 [2.22311058e-12 1.00000000e+00]
strat2 [2.34590125e-12 1.00000000e+00]
stage_test1 payoff vs stage_test2
1.0000000000071607
stage_test1 vs stage_test2 payoff
1.0000000000065468

=== Nash Equilibria (via NashPy) ===
Player 1: [0. 1.]
Player 2: [0. 1.]
Player1 payoff vs player2
1.0
Player1 vs player2 payoff
1.0
player 1 payoff vs stage_test2
1.0000000000093838
player 1 vs stage_test2 payoff
0.9999999999976543
stage_test1 payoff vs player2
0.9999999999977769
stage_test1 vs player2 payoff
1.0000000000088924


In [14]:
# ========================
# Problem Setup
# ========================

num_players = 2
n_strategies = 5
initial_strategy = np.array([0.2, 0.2,0.2, 0.2, 0.2] * num_players)

# Assumes U1 and U2 are defined elsewhere in your script
payoff_matrices = [U1, U2]

constraints = [
    {'type': 'eq', 'fun': lambda x: stochastic_strategy_constraint(x, num_players)},
    {'type': 'ineq', 'fun': lambda x: x}
]

# ========================
# Optimization
# ========================

res = minimize(
    fun=objective_function,
    x0=initial_strategy,
    args=(payoff_matrices, num_players),
    method='SLSQP',
    constraints=constraints
)

# ========================
# Results & Analysis
# ========================

if __name__ == '__main__':
    num_players   = 2
    n_strategies  = 5
    # A "stage_test" starting guess:
    initial_strat = np.array([0.2, 0.2,0.2, 0.2, 0.2] * num_players)

    # Build the payoff matrices U1, U2 for strategies 1–5
    strategies = [1,2,3,4,5]
    U1, U2 = np.zeros((5,5)), np.zeros((5,5))
    for i,s1 in enumerate(strategies):
        for j,s2 in enumerate(strategies):
            if abs(s1-s2)==1:
                if s1<s2:
                    U1[i,j], U2[i,j] = s1+s2, 0
                else:
                    U1[i,j], U2[i,j] = 0, s1+s2
            else:
                U1[i,j], U2[i,j] = s1, s2

    constraints = [
        {'type':'eq',   'fun': lambda x: stochastic_strategy_constraint(x, num_players)},
        {'type':'ineq', 'fun': lambda x: x}
    ]

    res = minimize(
        fun=objective_function,
        x0=initial_strat,
        args=([U1, U2], num_players),
        method='SLSQP',
        constraints=constraints
    )

    if res.success:
        x_sol  = res.x
        strat1 = x_sol[:n_strategies]   # Players1
        strat2 = x_sol[n_strategies:]   # player2

        print("✅ Optimization successful.\n")
        print_mixed_strategies(x_sol, num_players)
        print("\nStationarity residual:",
              objective_function(x_sol, [U1, U2], num_players))

        # Enumerate all Nash equilibria via NashPy
        game  = nash.Game(U1, U2)
        eqs   = list(game.vertex_enumeration())
        eq0   = eqs[0]  # pick the first equilibrium
        print("\n=== Nash Equilibria (via NashPy) ===")
        for p1, p2 in eqs:
            print(f"Player 1: {p1},\n Player 2: {p2}")

        x,y=pure_best_response(strat1, U1),pure_best_response(eq0[0], U1)
        print("\n stage_test payoff vs best_response"
            ,bilinear_payoff(strat1, U1, x))

        print("\n stage_test vs best_response payoff"
            ,bilinear_payoff(strat1, U2, x))

        print("\n Nash payoff vs best_response"
            ,bilinear_payoff(eq0[0], U1, y))
        print("\n Nash vs best_response payoff"
            ,bilinear_payoff(eq0[0], U2, y))


        print("\n stage_test payoff vs Nash",
              bilinear_payoff(strat1, U1, eq0[0]))
        print("\n Nash payoff vs stage_test",
              bilinear_payoff(strat1, U2, eq0[1]))
        print("\n stage_test payoff vs itself",
              bilinear_payoff(strat1, U1, strat1))
        print("\n Nash payoff vs itself",
              bilinear_payoff(eq0[0], U2, eq0[0]))
        # — Example: payoffs “against 4” and “against 3”
        pure_strats = np.eye(n_strategies)
        for idx in range(0,5):  # strategy 4 then 3
            print(f"\n— Against pure strategy {idx+1}:")
            # stage_test vs that pure
            print("  stage_test payoff:",
                  bilinear_payoff(strat1, U1, pure_strats[idx]))
            print("  pure payoff:",
                  bilinear_payoff(strat1, U2, pure_strats[idx]))
            print("\n")
            # Nash vs that pure
            print("  Nash   payoff:",
                  bilinear_payoff(eq0[0], U1, pure_strats[idx]))
            print("  pure payoff:",
                  bilinear_payoff(eq0[1], U2, pure_strats[idx]))

    else:
        print("❌ No feasible solution was found:", res.message)


✅ Optimization successful.

Player 1: [Fraction(27, 896), Fraction(440, 991), Fraction(101, 497), Fraction(292, 905), Fraction(0, 1)]
Player 2: [Fraction(28, 929), Fraction(218, 491), Fraction(101, 497), Fraction(151, 468), Fraction(0, 1)]

Stationarity residual: 80.89021558561456

=== Nash Equilibria (via NashPy) ===
Player 1: [1.34922937e-17 0.00000000e+00 4.44444444e-01 2.22222222e-01
 3.33333333e-01],
 Player 2: [1.34922937e-17 0.00000000e+00 4.44444444e-01 2.22222222e-01
 3.33333333e-01]

 stage_test payoff vs best_response 4.43165037918269

 stage_test vs best_response payoff 3.386739604228815

 Nash payoff vs best_response 5.0

 Nash vs best_response payoff 3.8888888888888897

 stage_test payoff vs Nash 3.5551729230140885

 Nash payoff vs stage_test 3.152105949286306

 stage_test payoff vs itself 2.8183899834115054

 Nash payoff vs itself 3.8888888888888893

— Against pure strategy 1:
  stage_test payoff: 1.930399095438621
  pure payoff: 1.8879908879728853


  Nash   payoff: 3.8